In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

# 1. Create DataFrame for Amsterdam Boroughs and their latitude and longitude

Create a dataframe for boroughs of Amsterdam with their latitude and longitude

In [6]:
# create a list of Amsterdam Borough's address, to be used with geopy.geocoders.Nominatim to get corresponding latitude and longitude values
centrum_address = 'Centrum, Amsterdam'
noord_address = 'Noord, Amsterdam'
west_address = 'West, Amsterdam'
nieuw_West_address = 'Nieuw-West, Amsterdam'
zuid_address = 'Zuid, Amsterdam'
oost_address = 'Oost, Amsterdam'
zuidoost_address = 'Zuidoost, Amsterdam'
westpoort_address ='Westpoort, Amsterdam'

borough_list = [centrum_address, noord_address, west_address, nieuw_West_address, zuid_address, oost_address, zuidoost_address, westpoort_address ]

In [7]:
# function to create a dataframe with columns listing boroughs and their corresponding latitudes and longitudes
def createDataframefromAddress(addresslist):
    df_boroughs = pd.DataFrame(columns=['Borough','Latitude','Longitude'])
    geolocator = Nominatim(user_agent="amborough_explorer")
    index = 0
    for address in addresslist:
        print(address)
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        # extract the borough from address
        b_name = address.split(',')[0]
        df_boroughs.loc[index] = [b_name, latitude, longitude]
        index = index+1

    return(df_boroughs) 
        

In [8]:
# create a pandas dataframe containing the list of boroughs alongwith their latitude and longitude

amsterdam_boroughs = createDataframefromAddress(borough_list)

Centrum, Amsterdam
Noord, Amsterdam
West, Amsterdam
Nieuw-West, Amsterdam
Zuid, Amsterdam
Oost, Amsterdam
Zuidoost, Amsterdam
Westpoort, Amsterdam


In [9]:
# check the dataframe
amsterdam_boroughs

,Borough,Latitude,Longitude
0,Centrum,52.373730,4.895691
1,Noord,52.372760,4.893604
2,West,52.372760,4.893604
3,Nieuw-West,52.363777,4.813812
4,Zuid,52.344470,4.878664
5,Oost,52.356608,4.930577
6,Zuidoost,52.310514,4.960695
7,Westpoort,52.411468,4.800573


# 2. Explore Venues near the Amsterdam Boroughs

In [10]:
CLIENT_ID = 'My Client ID' # Foursquare ID
CLIENT_SECRET = 'My Client Secret' # Foursquare Secret
VERSION = '20200601' # Foursquare API version
LIMIT = 100

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categorieId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoriesId)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
# we will be using the categoriesId parameter when we call the foursquare explore endpoint as a parameter to limit the results to venues which are of interest to us

arts_entertainment = '4d4b7104d754a06370d81259'
event = '4d4b7105d754a06373d81259'
food = '4d4b7105d754a06374d81259'
nightlife = '4d4b7105d754a06376d81259'
shop_service = '4d4b7105d754a06378d81259'

categoriesId = '{},{},{},{},{}'.format(arts_entertainment, event, food, nightlife, shop_service)


In [14]:
# get the venues from foursquare
amsterdam_venues = getNearbyVenues(names=amsterdam_boroughs['Borough'],
                                   latitudes=amsterdam_boroughs['Latitude'],
                                   longitudes=amsterdam_boroughs['Longitude']
                                  )

Centrum
Noord
West
Nieuw-West
Zuid
Oost
Zuidoost
Westpoort


In [15]:
# check the dataframe built from the results of the foursquare calls
print(amsterdam_venues.shape)
amsterdam_venues.head()

(324, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centrum,52.37373,4.895691,De Koffieschenkerij,52.374043,4.898427,Coffee Shop
1,Centrum,52.37373,4.895691,Dam,52.372824,4.893702,Plaza
2,Centrum,52.37373,4.895691,Wynand Fockink,52.372301,4.895253,Liquor Store
3,Centrum,52.37373,4.895691,Old Amsterdam Cheese Store,52.374862,4.895301,Cheese Shop
4,Centrum,52.37373,4.895691,Red Light Radio,52.374626,4.897840,Music Venue


In [16]:
amsterdam_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Centrum,78,78,78,78,78,78
Nieuw-West,5,5,5,5,5,5
Noord,82,82,82,82,82,82
Oost,44,44,44,44,44,44
West,82,82,82,82,82,82
Westpoort,1,1,1,1,1,1
Zuid,23,23,23,23,23,23
Zuidoost,9,9,9,9,9,9


In [17]:
print('There are {} uniques categories.'.format(len(amsterdam_venues['Venue Category'].unique())))

There are 96 uniques categories.


# 3. Analyze Each Borough

In [18]:
# one hot encoding
amsterdam_onehot = pd.get_dummies(amsterdam_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
amsterdam_onehot['Borough'] = amsterdam_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [amsterdam_onehot.columns[-1]] + list(amsterdam_onehot.columns[:-1])
amsterdam_onehot = amsterdam_onehot[fixed_columns]

amsterdam_onehot.head()

,Borough,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bar,Beer Garden,Beer Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Stop,Butcher,Café,Campground,Canal,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Deli / Bodega,Department Store,Dessert Shop,Dive Bar,Drugstore,Fish Market,Flea Market,French Restaurant,Friterie,Gastropub,Gay Bar,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Heliport,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Lake,Lawyer,Liquor Store,Malay Restaurant,Marijuana Dispensary,Mediterranean Restaurant,Museum,Music School,Music Venue,North Indian Restaurant,Optical Shop,Organic Grocery,Palace,Park,Plaza,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Soccer Field,Spa,Spiritual Center,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tea Room,Thai Restaurant,Theater,Track,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store
0,Centrum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Centrum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Centrum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Centrum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Centrum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
amsterdam_onehot.shape

(324, 97)

In [20]:
amsterdam_grouped = amsterdam_onehot.groupby('Borough').mean().reset_index()
amsterdam_grouped

,Borough,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bar,Beer Garden,Beer Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Stop,Butcher,Café,Campground,Canal,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Deli / Bodega,Department Store,Dessert Shop,Dive Bar,Drugstore,Fish Market,Flea Market,French Restaurant,Friterie,Gastropub,Gay Bar,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Heliport,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Lake,Lawyer,Liquor Store,Malay Restaurant,Marijuana Dispensary,Mediterranean Restaurant,Museum,Music School,Music Venue,North Indian Restaurant,Optical Shop,Organic Grocery,Palace,Park,Plaza,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Soccer Field,Spa,Spiritual Center,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tea Room,Thai Restaurant,Theater,Track,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store
0,Centrum,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.012821,0.064103,0.0,0.012821,0.000000,0.012821,0.012821,0.012821,0.000000,0.025641,0.000000,0.012821,0.076923,0.0,0.000000,0.025641,0.012821,0.012821,0.025641,0.038462,0.038462,0.051282,0.000000,0.000000,0.038462,0.012821,0.012821,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.012821,0.012821,0.000000,0.012821,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.064103,0.012821,0.012821,0.012821,0.000000,0.012821,0.000000,0.0,0.000000,0.012821,0.000000,0.000000,0.025641,0.012821,0.000000,0.025641,0.000000,0.000000,0.000000,0.012821,0.000000,0.025641,0.025641,0.012821,0.000000,0.025641,0.012821,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.012821,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821
1,Nieuw-West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Noord,0.012195,0.024390,0.012195,0.012195,0.000000,0.000000,0.024390,0.060976,0.0,0.012195,0.000000,0.024390,0.012195,0.000000,0.000000,0.012195,0.000000,0.012195,0.024390,0.0,0.012195,0.012195,0.024390,0.012195,0.012195,0.036585,0.036585,0.060976,0.000000,0.012195,0.024390,0.012195,0.024390,0.012195,0.000000,0.000000,0.000000,0.012195,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,0.000000,0.0,0.000000,0.0,0.012195,0.073171,0.000000,0.012195,0.012195,0.012195,0.000000,0.000000,0.0,0.000000,0.012195,0.012195,0.012195,0.012195,0.024390,0.000000,0.012195,0.000000,0.000000,0.000000,0.012195,0.000000,0.024390,0.012195,0.012195,0.012195,0.024390,0.012195,0.000000,0.012195,0.012195,0.000000,0.012195,0.000000,0.012195,0.000000,0.024390,0.000000,0.000000,0.012195,0.012195,0.012195,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000
3,Oost,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.045455,0.045455,0.0,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.022727,0.000000,0.000000,0.022727,0.0,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.00

In [21]:
num_top_venues = 10

for hood in amsterdam_grouped['Borough']:
    print("----"+hood+"----")
    temp = amsterdam_grouped[amsterdam_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Centrum----
                    venue  freq
0                    Café  0.08
1                   Hotel  0.06
2                     Bar  0.06
3             Coffee Shop  0.05
4           Deli / Bodega  0.04
5       French Restaurant  0.04
6            Cocktail Bar  0.04
7          Clothing Store  0.04
8  Argentinian Restaurant  0.04
9                  Church  0.03


----Nieuw-West----
             venue  freq
0  Harbor / Marina   0.2
1             Lake   0.2
2      Beer Garden   0.2
3       Campground   0.2
4         Bus Stop   0.2
5           Arcade   0.0
6     Music School   0.0
7            Plaza   0.0
8             Park   0.0
9           Palace   0.0


----Noord----
                    venue  freq
0                   Hotel  0.07
1                     Bar  0.06
2             Coffee Shop  0.06
3          Clothing Store  0.04
4            Cocktail Bar  0.04
5  Argentinian Restaurant  0.02
6                  Museum  0.02
7                   Plaza  0.02
8                    Café  0.02


In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = amsterdam_grouped['Borough']

for ind in np.arange(amsterdam_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(amsterdam_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centrum,Café,Bar,Hotel,Coffee Shop,Deli / Bodega,Cocktail Bar,Clothing Store,French Restaurant,Argentinian Restaurant,Music Venue
1,Nieuw-West,Harbor / Marina,Campground,Bus Stop,Beer Garden,Lake,Drugstore,Coffee Shop,Comfort Food Restaurant,Comic Shop,Deli / Bodega
2,Noord,Hotel,Bar,Coffee Shop,Cocktail Bar,Clothing Store,Museum,Bakery,Plaza,Café,Chinese Restaurant
3,Oost,French Restaurant,Coffee Shop,Turkish Restaurant,Restaurant,Gym / Fitness Center,Bakery,Bar,Liquor Store,North Indian Restaurant,Japanese Restaurant
4,West,Hotel,Bar,Coffee Shop,Cocktail Bar,Clothing Store,Museum,Bakery,Plaza,Café,Chinese Restaurant


# 4. Cluster Boroughs

In [24]:
# set number of clusters
kclusters = 5

amsterdam_grouped_clustering = amsterdam_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(amsterdam_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 1, 4, 2], dtype=int32)

In [25]:
# add clustering labels
borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

amsterdam_merged = amsterdam_boroughs

# merge amsterdam_grouped with amsterdam_boroughs to add latitude/longitude for each neighborhood
amsterdam_merged = amsterdam_merged.join(borough_venues_sorted.set_index('Borough'), on='Borough')


In [26]:
amsterdam_merged.head(10)

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centrum,52.373730,4.895691,0,Café,Bar,Hotel,Coffee Shop,Deli / Bodega,Cocktail Bar,Clothing Store,French Restaurant,Argentinian Restaurant,Music Venue
1,Noord,52.372760,4.893604,0,Hotel,Bar,Coffee Shop,Cocktail Bar,Clothing Store,Museum,Bakery,Plaza,Café,Chinese Restaurant
2,West,52.372760,4.893604,0,Hotel,Bar,Coffee Shop,Cocktail Bar,Clothing Store,Museum,Bakery,Plaza,Café,Chinese Restaurant
3,Nieuw-West,52.363777,4.813812,3,Harbor / Marina,Campground,Bus Stop,Beer Garden,Lake,Drugstore,Coffee Shop,Comfort Food Restaurant,Comic Shop,Deli / Bodega
4,Zuid,52.344470,4.878664,4,Park,Tram Station,Restaurant,Bistro,Chinese Restaurant,Department Store,Lawyer,Café,Bus Stop,Bridal Shop
5,Oost,52.356608,4.930577,0,French Restaurant,Coffee Shop,Turkish Restaurant,Restaurant,Gym / Fitness Center,Bakery,Bar,Liquor Store,North Indian Restaurant,Japanese Restaurant
6,Zuidoost,52.310514,4.960695,2,Park,Soccer Field,Track,Theater,Athletics & Sports,Drugstore,Dive Bar,Church,Clothing Store,Cocktail Bar
7,Westpoort,52.411468,4.800573,1,Heliport,Vegetarian / Vegan Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Deli / Bodega,Department Store,Dessert Shop


In [27]:
# Get latitude and longitude of Amsterdam
address = 'Amsterdam, Netherlands'

geolocator = Nominatim(user_agent="am_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(amsterdam_merged['Latitude'], amsterdam_merged['Longitude'], amsterdam_merged['Borough'], amsterdam_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examine Clusters

**Cluster 1 - (Centrum, Noord, West and Oost)**

In [29]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 0, amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,52.373730,Bar,Hotel,Coffee Shop,Deli / Bodega,Cocktail Bar,Clothing Store,French Restaurant,Argentinian Restaurant,Music Venue
1,52.372760,Bar,Coffee Shop,Cocktail Bar,Clothing Store,Museum,Bakery,Plaza,Café,Chinese Restaurant
2,52.372760,Bar,Coffee Shop,Cocktail Bar,Clothing Store,Museum,Bakery,Plaza,Café,Chinese Restaurant
5,52.356608,Coffee Shop,Turkish Restaurant,Restaurant,Gym / Fitness Center,Bakery,Bar,Liquor Store,North Indian Restaurant,Japanese Restaurant


**CLuster 2 - Westport**

In [30]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 1, amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,52.411468,Vegetarian / Vegan Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Deli / Bodega,Department Store,Dessert Shop


**Cluster 3 - Zuidoost**

In [31]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 2, amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,52.310514,Soccer Field,Track,Theater,Athletics & Sports,Drugstore,Dive Bar,Church,Clothing Store,Cocktail Bar


**Cluster 4 - Nieuw-West**

In [32]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 3, amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,52.363777,Campground,Bus Stop,Beer Garden,Lake,Drugstore,Coffee Shop,Comfort Food Restaurant,Comic Shop,Deli / Bodega


**Cluster 5 - Zuid**

In [33]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 4, amsterdam_merged.columns[[1] + list(range(5, amsterdam_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,52.34447,Tram Station,Restaurant,Bistro,Chinese Restaurant,Department Store,Lawyer,Café,Bus Stop,Bridal Shop
